In [ ]:
#query for efi2 deficit for urinary system disease, using updated code for efalls deficits
 DROP TABLE `yhcr-prd-phm-bia-core.CB_MYSPACE_MK.temp_cb_2172_efalls_urinary_disease` ;
 CREATE TABLE `yhcr-prd-phm-bia-core.CB_2172.cb_2172_efalls_urinary_disease` as (
#bring all snoemd and ctv3codes for care home cohort in primary into the cte
with a as (
  select
  person_id,
  ctv3code,
  snomedcode,
  dateevent
  from `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  where person_id in(select person_id from `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic`)),
  #join care home entry date from msater table
  a_master as (
    select
    a.*,
    j.first_episodestartdate
    from a
    left join `yhcr-prd-phm-bia-core.CB_2172.temp_cb_2172_ch_cohort_demographic` j on
    a.person_id = j.person_id),
#filter for Urinary system disease problems snomed codes
  b as (
    select
    person_id,
    case
    when snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Urinary system disease' and TimeConstraintyears is null) AND dateevent < first_episodestartdate then 'true'
    when ctv3code in(select CTV3 from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Urinary system disease'  and TimeConstraintyears is null) AND dateevent < first_episodestartdate then 'true'
    when snomedcode in(select SNOMEDCT_CONCEPTID from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Urinary system disease' and TimeConstraintyears is not null) AND dateevent < first_episodestartdate and (datetime_diff(first_episodestartdate,dateevent,year) <= 5) then 'true'
    when ctv3code in(select CTV3 from `yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_EFI2_Codelist` where deficit = 'Urinary system disease'  and TimeConstraintyears is not null) AND dateevent < first_episodestartdate and (datetime_diff(first_episodestartdate,dateevent,year) <= 5) then 'true' end as urinary_disease
    from a_master)

#apply dummy variable
    select
     distinct person_id,
    urinary_disease
    from b
    where urinary_disease = 'true'
    order by person_id)
  



